In [1]:
import pandas as pd
import re

In [2]:
salary_min_field = "WAGE_OFFER_FROM_9089"
salary_max_field = "WAGE_OFFER_TO_9089"

salary_freq_field = "WAGE_OFFER_UNIT_OF_PAY_9089"

# salary_field = "PW_AMOUNT_9089"
# salary_freq_field = "PW_UNIT_OF_PAY_9089"

In [3]:
job_zone_df = pd.read_csv('All_Job_Zones.csv')

In [4]:
job_zone_df['Full_Code'] = job_zone_df['Code']
job_zone_df['Code'] = job_zone_df['Code'].str.split(".").str[0]
job_zone_df.drop_duplicates(subset=['Code'], keep='first', inplace=True)
print(job_zone_df.head(10))

    Job Zone     Code                                 Occupation   Full_Code
0          5  11-1011                           Chief Executives  11-1011.00
2          4  11-1021            General and Operations Managers  11-1021.00
3          4  11-1031                                Legislators  11-1031.00
4          4  11-2011        Advertising and Promotions Managers  11-2011.00
5          4  11-2021                         Marketing Managers  11-2021.00
6          4  11-2022                             Sales Managers  11-2022.00
7          4  11-2031  Public Relations and Fundraising Managers  11-2031.00
8          3  11-3011           Administrative Services Managers  11-3011.00
9          4  11-3021  Computer and Information Systems Managers  11-3021.00
10         5  11-3031                 Treasurers and Controllers  11-3031.01


In [ ]:
perm_data_df = pd.read_excel('PERM_Disclosure_Data_FY17.xlsx', sheet_name='FY17_EOQ4_PERM_DataDump')


In [ ]:
print(list(perm_data_df))

In [ ]:
groups = perm_data_df.groupby('CASE_STATUS')

status_list = ["Withdrawn", "Certified-Expired", "Certified", "Denied"]
count_by_status_list = []
for status in status_list:
    adict = dict()
    adict['status'] = status
    adict['count'] = groups.get_group(status)['CASE_NUMBER'].count()
    count_by_status_list.append(adict)

count_by_status_df = pd.DataFrame(count_by_status_list)
print(count_by_status_df)


In [14]:
perm_data_filtered_df = perm_data_df[(perm_data_df['CASE_STATUS'] == 'Certified') | (perm_data_df['CASE_STATUS'] == 'Certified-Expired')]
perm_data_filtered_df = perm_data_filtered_df[~(perm_data_filtered_df[salary_freq_field].isnull())]

for salary_field in [salary_min_field, salary_max_field]:
    perm_data_filtered_df[salary_field] = perm_data_filtered_df[salary_field].str.replace(",", "")
    perm_data_filtered_df[salary_field] = pd.to_numeric(perm_data_filtered_df[salary_field], errors='coerce')

hour = 40 * 52
week = 52
biweekly = 26
month = 12

def annualize_min(x):
    offer = x[salary_min_field]
    offer_unit = x[salary_freq_field]
    return annualize(offer, offer_unit)
    
def annualize_max(x):
    offer = x[salary_max_field]
    if pd.isnull(offer):
        offer = x[salary_min_field]
        
    offer_unit = x[salary_freq_field]
    return annualize(offer, offer_unit)

def annualize(offer, offer_unit):
    annualized = offer
    
    if offer_unit == 'Hour':
        annualized = offer * hour
    elif offer_unit == 'Week':
        annualized = offer * week
    elif offer_unit == 'Bi-Weekly':
        annualized = offer * biweekly
    elif offer_unit == 'Month':
        annualized = offer * month
    
    return annualized

def annualize_mean(x):
    return (x['annualized_min'] + x['annualized_max']) / 2

perm_data_filtered_df['annualized_min'] = perm_data_filtered_df.apply(annualize_min, axis=1)
perm_data_filtered_df['annualized_max'] = perm_data_filtered_df.apply(annualize_max, axis=1)
perm_data_filtered_df['annualized'] = perm_data_filtered_df.apply(annualize_mean, axis=1)

print(perm_data_filtered_df[['CASE_NUMBER', salary_min_field, salary_max_field, 'annualized_min', 'annualized_max', 'annualized']].head(10))

      CASE_NUMBER  WAGE_OFFER_FROM_9089  WAGE_OFFER_TO_9089  annualized_min  \
11  A-16144-13619               68078.0                 NaN         68078.0   
12  A-16202-34615               36566.0                 NaN         36566.0   
13  A-16211-37294              112632.0                 NaN        112632.0   
14  A-16097-93778               74000.0                 NaN         74000.0   
15  A-16096-92998               74000.0                 NaN         74000.0   
16  A-16189-30349              150500.0            173075.0        150500.0   
17  A-16070-82635              114400.0                 NaN        114400.0   
18  A-16033-67970               67995.0                 NaN         67995.0   
19  A-16167-22350              148500.0            168500.0        148500.0   
20  A-16167-22359              160000.0            173056.0        160000.0   

    annualized_max  annualized  
11         68078.0     68078.0  
12         36566.0     36566.0  
13        112632.0    112632.0 

In [27]:
merged_df = pd.merge(perm_data_filtered_df,job_zone_df, how='left', left_on='PW_SOC_CODE', right_on='Code')
merged_df = merged_df[((merged_df['annualized'] >= 13624) & (merged_df['annualized'] < 300000))]
job_zone_list = []

india_df = merged_df[(merged_df['FW_INFO_BIRTH_COUNTRY'] == 'INDIA')]
job_zone_dict = dict()
job_zone_dict['Country'] = 'India'
job_zone_dict['JobZone_Mean'] = round(india_df['Job Zone'].mean(), 2)
job_zone_dict['JobZone_Median'] = round(india_df['Job Zone'].median(), 2)
job_zone_dict['Salary_Mean'] = round(india_df['annualized'].mean(), 2)
job_zone_dict['Salary_STDEV'] = round(india_df['annualized'].std(),2)
job_zone_dict['Salary_Median'] = round(india_df['annualized'].median(), 2)
job_zone_list.append(job_zone_dict)

china_df = merged_df[(merged_df['FW_INFO_BIRTH_COUNTRY'] == 'CHINA')]
job_zone_dict = dict()
job_zone_dict['Country'] = 'China'
job_zone_dict['JobZone_Mean'] = round(china_df['Job Zone'].mean(), 2)
job_zone_dict['JobZone_Median'] = round(china_df['Job Zone'].median(), 2)
job_zone_dict['Salary_Mean'] = round(china_df['annualized'].mean(), 2)
job_zone_dict['Salary_STDEV'] = round(china_df['annualized'].std(),2)
job_zone_dict['Salary_Median'] = round(china_df['annualized'].median(), 2)
job_zone_list.append(job_zone_dict)

indochina_df = merged_df[((merged_df['FW_INFO_BIRTH_COUNTRY'] == 'CHINA') | (merged_df['FW_INFO_BIRTH_COUNTRY'] == 'INDIA'))]
job_zone_dict = dict()
job_zone_dict['Country'] = 'India and China'
job_zone_dict['JobZone_Mean'] = round(indochina_df['Job Zone'].mean(), 2)
job_zone_dict['JobZone_Median'] = round(indochina_df['Job Zone'].median(), 2)
job_zone_dict['Salary_Mean'] = round(indochina_df['annualized'].mean(), 2)
job_zone_dict['Salary_STDEV'] = round(indochina_df['annualized'].std(),2)
job_zone_dict['Salary_Median'] = round(indochina_df['annualized'].median(), 2)
job_zone_list.append(job_zone_dict)

row_minus_india_df = merged_df[~(merged_df['FW_INFO_BIRTH_COUNTRY'] == 'INDIA')]
job_zone_dict = dict()
job_zone_dict['Country'] = 'ROW Minus India'
job_zone_dict['JobZone_Mean'] = round(row_minus_india_df['Job Zone'].mean(), 2)
job_zone_dict['JobZone_Median'] = round(row_minus_india_df['Job Zone'].median(), 2)
job_zone_dict['Salary_Mean'] = round(row_minus_india_df['annualized'].mean(), 2)
job_zone_dict['Salary_STDEV'] = round(row_minus_india_df['annualized'].std(),2)
job_zone_dict['Salary_Median'] = round(row_minus_india_df['annualized'].median(), 2)
job_zone_list.append(job_zone_dict)

row_minus_indochina_df = merged_df[~((merged_df['FW_INFO_BIRTH_COUNTRY'] == 'INDIA') | (merged_df['FW_INFO_BIRTH_COUNTRY'] == 'CHINA'))]
job_zone_dict = dict()
job_zone_dict['Country'] = 'ROW Minus India, China'
job_zone_dict['JobZone_Mean'] = round(row_minus_indochina_df['Job Zone'].mean(), 2)
job_zone_dict['JobZone_Median'] = round(row_minus_indochina_df['Job Zone'].median(), 2)
job_zone_dict['Salary_STDEV'] = round(row_minus_indochina_df['annualized'].std(),2)
job_zone_dict['Salary_Mean'] = round(row_minus_indochina_df['annualized'].mean(), 2)
job_zone_dict['Salary_Median'] = round(row_minus_indochina_df['annualized'].median(), 2)
job_zone_list.append(job_zone_dict)

stats_df = pd.DataFrame(job_zone_list)
print(stats_df)


                  Country  JobZone_Mean  JobZone_Median  Salary_Mean  \
0                   India          4.05             4.0    110616.35   
1                   China          3.96             4.0    105431.17   
2         India and China          4.03             4.0    109852.26   
3         ROW Minus India          3.71             4.0     90335.04   
4  ROW Minus India, China          3.65             4.0     86281.56   

   Salary_STDEV  Salary_Median  
0      30009.59       107363.5  
1      40600.95       112467.8  
2      31845.43       108000.0  
3      51417.93        87069.0  
4      53230.70        80000.0  


In [68]:

stats_df.to_csv('job_zone_salary_analysis.csv')